In [1]:
pip install torch transformers pandas accelerate

In [3]:
import torch
import pandas as pd
from transformers import BloomTokenizerFast, BloomForCausalLM
import random
import textwrap

# --- Configuration ---
FILE_NAME = "PoetryFoundationData_Cleaned.csv"
MODEL_NAME = "bigscience/bloom-560m"
USER_PROMPT = "A serene morning in the mountains."

# Generation parameters
MAX_NEW_TOKENS = 120
TEMPERATURE = 0.9
NUM_RETURN_SEQUENCES = 3
NUM_EXAMPLES_FROM_DATASET = 3 # The 'few-shot' part

# --- 1. Load Data and Construct Few-Shot Prompt ---
print(f"Loading data from {FILE_NAME}...")
try:
    # Load the cleaned poetry data
    df = pd.read_csv(FILE_NAME)

    # Filter for the main text column and sample a few poems
    sample_poems = df['Cleaned_Poem'].dropna().sample(NUM_EXAMPLES_FROM_DATASET, random_state=42)

    # Start building the ICL prompt with system instructions
    prompt_template = "Below are examples of modern poetry. Study their style and cadence, then complete the final poem based on the last prompt.\n\n"

    # Add the sampled poems as examples
    for i, poem in enumerate(sample_poems):
        # Format the poem with line breaks to mimic natural poetry structure
        formatted_poem = textwrap.fill(poem, width=80)

        prompt_template += f"### Example Poem {i+1} (Source: Poetry Foundation Dataset) ###\n"
        prompt_template += f"{formatted_poem}\n\n"

    # Add the final prompt that the model must complete
    prompt_template += f"### NEW POEM (Prompt: {USER_PROMPT}) ###\n"
    prompt_template += f"{USER_PROMPT}\n"

    final_prompt = prompt_template

    print(f"Few-shot prompt constructed with {NUM_EXAMPLES_FROM_DATASET} examples.")

except FileNotFoundError:
    print(f"Error: The file '{FILE_NAME}' was not found.")
    print("Using a placeholder poem to continue the demonstration.")
    # Fallback in case the file is not correctly accessible during execution
    final_prompt = (
        "### Example Poem 1 ###\n"
        "The entrance at the back of the complex led onto a road, where an upended couch\n"
        "tilted into a ditch and a washing machine gleamed avocado beneath pine needles.\n"
        "But doesn't longing clarify the body?\n\n"
        f"### NEW POEM (Prompt: {USER_PROMPT}) ###\n{USER_PROMPT}\n"
    )
except Exception as e:
    print(f"An error occurred during file processing: {e}")
    print("Using a basic prompt for demonstration.")
    final_prompt = USER_PROMPT


# --- 2. Load the BLOOM-560M Model and Tokenizer ---
print("\nLoading BLOOM-560M model and tokenizer...")
try:
    tokenizer = BloomTokenizerFast.from_pretrained(MODEL_NAME)
    model = BloomForCausalLM.from_pretrained(MODEL_NAME)
    print("Model loaded successfully.")

except Exception as e:
    print(f"An error occurred while loading the model: {e}")
    print("Please ensure you have the 'transformers' and 'torch' libraries installed.")
    exit()

# --- 3. Generate the Poetry ---
print("\nGenerating poetry using the dataset-guided prompt...")

# Encode the prompt text
input_ids = tokenizer.encode(final_prompt, return_tensors='pt')
prompt_length = len(input_ids[0])

# Generate the text
output = model.generate(
    input_ids,
    max_length=prompt_length + MAX_NEW_TOKENS,
    temperature=TEMPERATURE,
    num_return_sequences=NUM_RETURN_SEQUENCES,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    pad_token_id=tokenizer.eos_token_id
)
print("Generation complete.")

# --- 4. Decode and Display the Results ---
print("\n" + "="*50)
print("--- GENERATED POEMS (Guided by PoetryFoundation Data) ---")
print("="*50)

for i, sequence in enumerate(output):
    # Decode the full generated sequence
    generated_text = tokenizer.decode(sequence, skip_special_tokens=True)

    # Find the end of the input prompt in the generated text to only display the new poem
    # This step ensures we only output what the model generated, not the few-shot examples
    start_index = generated_text.rfind(USER_PROMPT) + len(USER_PROMPT)
    new_poem = generated_text[start_index:].strip()

    print(f"\n[Poem {i+1}]")
    print("-" * 20)
    print(USER_PROMPT) # Print the user prompt again for context
    print(new_poem)
    print("-" * 20)

Loading data from PoetryFoundationData_Cleaned.csv...
Few-shot prompt constructed with 3 examples.

Loading BLOOM-560M model and tokenizer...
Model loaded successfully.

Generating poetry using the dataset-guided prompt...
Generation complete.

--- GENERATED POEMS (Guided by PoetryFoundation Data) ---

[Poem 1]
--------------------
A serene morning in the mountains.
### Example Text 1 (Source: Poetry Foundation Dataset) ###

She had the jitters

She had the flu

She showed up late

She missed her cue

She kicked the director

She screamed at the crew

And tripped on a prop

And fell in some goo

And ripped her costume

A place or two Then she forgot

A line she knew

And went “Meow”

Instead of “Moo”

She heard ‘em giggle

She heard ‘em boo

The programs sailed The popcorn flew As she stomped off
--------------------

[Poem 2]
--------------------
A serene morning in the mountains.
### Example Poem 1 (Source: Poetry Foundation Dataset) ###
How, in a few days, The sun had turned its bri

In [5]:
# Required libraries for BLOOM, Accelerate (for training efficiency),
# Datasets (for data handling), and Pandas (for CSV loading)
!pip install torch transformers datasets accelerate pandas
import pandas as pd
from datasets import Dataset
from transformers import (
    BloomTokenizerFast,
    BloomForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
import torch
import os
import logging
import random

# --- Configuration ---
# ⚠️ Ensure this file is in the same directory as your script
FILE_PATH = "PoetryFoundationData_Cleaned.csv"
TEXT_COLUMN = "Cleaned_Poem" # The column containing the poem text
MODEL_NAME = "bigscience/bloom-560m"
OUTPUT_DIR = "./bloom_poetry_finetuned"
BLOCK_SIZE = 128 # Max sequence length for training blocks (adjust based on GPU memory)
TRAIN_RATIO = 0.90

# Training Hyperparameters (Adjust these based on your GPU and desired training time)
LEARNING_RATE = 2e-5
EPOCHS = 3
BATCH_SIZE = 4 # Use a small batch size for a 560M parameter model on consumer GPUs
WEIGHT_DECAY = 0.01

# --- 2. Data Preparation ---

def load_and_prepare_data(file_path, text_column, train_ratio):
    """Loads CSV, extracts text, and splits into training/validation datasets."""
    print(f"Loading data from {file_path}...")

    # 1. Load data using pandas
    df = pd.read_csv(file_path).dropna(subset=[text_column])

    # 2. Convert to Hugging Face Dataset format
    dataset = Dataset.from_pandas(df[[text_column]])

    # 3. Split data
    split_datasets = dataset.train_test_split(test_size=1.0 - train_ratio, seed=42)
    return split_datasets['train'], split_datasets['test']

def tokenize_function(examples, tokenizer):
    """Tokenizes the text data."""
    # We use padding=False and truncation=True here
    return tokenizer(examples[TEXT_COLUMN], truncation=True)

def group_texts(examples):
    """Concatenates all texts and splits them into fixed-size blocks (causal language modeling)."""
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])

    # Drop the last remainder block if it's smaller than BLOCK_SIZE
    total_length = (total_length // BLOCK_SIZE) * BLOCK_SIZE

    # Split by BLOCK_SIZE
    result = {
        k: [t[i : i + BLOCK_SIZE] for i in range(0, total_length, BLOCK_SIZE)]
        for k, t in concatenated_examples.items()
    }
    # Labels are needed for Causal Language Modeling; they are simply the input IDs
    result["labels"] = result["input_ids"].copy()
    return result

# --- 3. Training Function ---

def fine_tune_model(train_dataset, eval_dataset, tokenizer, model):
    """Sets up the Trainer and runs the fine-tuning process."""

    # The data collator prepares batches of data and applies causal language modeling logic (labels = inputs shifted)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        overwrite_output_dir=True,
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        learning_rate=LEARNING_RATE,
        weight_decay=WEIGHT_DECAY,
        logging_dir='./logs',
        logging_steps=50,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        fp16=torch.cuda.is_available() # Use 16-bit precision if GPU is available
    )

    # Initialize the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    print("\n" + "="*50)
    print("STARTING FINE-TUNING...")
    print("="*50)

    # Run training
    trainer.train()

    # Save the fine-tuned model
    trainer.save_model(os.path.join(OUTPUT_DIR, "final_model"))
    tokenizer.save_pretrained(os.path.join(OUTPUT_DIR, "final_model"))
    print(f"\nFine-tuned model saved to: {os.path.join(OUTPUT_DIR, 'final_model')}")

# --- 4. Generation Function ---

def generate_poetry(prompt, tokenizer, model, num_sequences=3, max_length=150):
    """Generates text using the fine-tuned model."""

    # Ensure model is in evaluation mode
    model.eval()

    # Encode the prompt
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(model.device)
    prompt_length = len(input_ids[0])

    print(f"\n--- Generating {num_sequences} Poems ---")

    # Use torch.no_grad() for inference
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=prompt_length + max_length,
            num_return_sequences=num_sequences,
            temperature=0.95,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            pad_token_id=tokenizer.eos_token_id
        )

    for i, sequence in enumerate(output):
        generated_text = tokenizer.decode(sequence, skip_special_tokens=True)

        # Only show the text generated *after* the prompt
        # We find the end of the prompt and strip the rest
        new_text = generated_text[len(prompt):].strip()

        print(f"\n[Poem {i+1}]")
        print("-" * 30)
        print(f"Prompt: {prompt}")
        print(new_text)
        print("-" * 30)

# --- 5. Main Execution Block ---

if __name__ == "__main__":
    # Set up device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # A. Load Base Model and Tokenizer
    tokenizer = BloomTokenizerFast.from_pretrained(MODEL_NAME)
    # Set padding token for batch processing/generation (often missing for CLM models)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model = BloomForCausalLM.from_pretrained(MODEL_NAME).to(device)

    # B. Load and Preprocess Data
    train_ds, eval_ds = load_and_prepare_data(FILE_PATH, TEXT_COLUMN, TRAIN_RATIO)

    # Tokenize the dataset
    print("Tokenizing and grouping data...")
    tokenized_train_ds = train_ds.map(lambda x: tokenize_function(x, tokenizer), batched=True, num_proc=4, remove_columns=train_ds.column_names)
    tokenized_eval_ds = eval_ds.map(lambda x: tokenize_function(x, tokenizer), batched=True, num_proc=4, remove_columns=eval_ds.column_names)

    # Group texts into fixed-size blocks (the core CLM data preparation)
    lm_train_dataset = tokenized_train_ds.map(group_texts, batched=True, batch_size=1000, num_proc=4)
    lm_eval_dataset = tokenized_eval_ds.map(group_texts, batched=True, batch_size=1000, num_proc=4)

    print(f"Total training blocks: {len(lm_train_dataset)}")

    # C. Run Fine-Tuning
    # fine_tune_model(lm_train_dataset, lm_eval_dataset, tokenizer, model)

    # NOTE: Since actual training cannot be run here, the line above is commented out.
    # To run this code, you MUST uncomment the fine_tune_model line.
    print("\n*** TRAINING STEP SKIPPED: UNCOMMENT LINE 187 TO RUN FINE-TUNING ***")

    # D. Load the FINE-TUNED Model (or base model if training is skipped) and Generate
    try:
        # Attempt to load the fine-tuned model (if training was run previously)
        final_model_path = os.path.join(OUTPUT_DIR, "final_model")
        print(f"\nAttempting to load fine-tuned model from: {final_model_path}")
        finetuned_model = BloomForCausalLM.from_pretrained(final_model_path).to(device)
        finetuned_tokenizer = BloomTokenizerFast.from_pretrained(final_model_path)
    except:
        # Fallback to the base model if the fine-tuned model is not found
        print("Fine-tuned model not found. Using the base BLOOM-560M model for generation.")
        finetuned_model = model
        finetuned_tokenizer = tokenizer


    # E. Prompt and Generate
    user_input_prompt = input("\nEnter your prompt for the poem: ")

    # The prompt structure needs to be simple for a model trained with CLM
    # A simple instruction/prompt is concatenated with the text it should continue
    generation_prompt = f"A poem based on the prompt: '{user_input_prompt}'.\n\nPoem: "

    generate_poetry(
        prompt=generation_prompt,
        tokenizer=finetuned_tokenizer,
        model=finetuned_model,
        max_length=150
    )

Using device: cpu
Loading data from PoetryFoundationData_Cleaned.csv...
Tokenizing and grouping data...


Map (num_proc=4):   0%|          | 0/12306 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map (num_proc=4):   0%|          | 0/1368 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map (num_proc=4):   0%|          | 0/12306 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1368 [00:00<?, ? examples/s]

Total training blocks: 32817

*** TRAINING STEP SKIPPED: UNCOMMENT LINE 187 TO RUN FINE-TUNING ***

Attempting to load fine-tuned model from: ./bloom_poetry_finetuned/final_model
Fine-tuned model not found. Using the base BLOOM-560M model for generation.

Enter your prompt for the poem: A serene morning in the mountains.

--- Generating 3 Poems ---

[Poem 1]
------------------------------
Prompt: A poem based on the prompt: 'A serene morning in the mountains.'.

Poem: 
The Beaten One
Scent:

A melody of a gentle spring morning, whose morning I have
remembered.--(From the Poem to the Beaten One, by F. Gaskell)
I think it is true that my little girl had heard
the voice of her mother say something like this,
'Heaven! Heaven! my son! Heaven! I have been thinking
of him, my dear son, in the mountains, when I heard the
voice of the mother speak,--oh! he is gone away
where I left him!'

In the early years of this century the French poet
Léon Gérôme had written a sermon on the subject of
the b